In [1]:
import cv2
import numpy as np
import os
from PIL import Image
import torch
import json
import csv
from collections import defaultdict, deque
from facenet_pytorch import MTCNN
from deep_sort_realtime.deepsort_tracker import DeepSort
from insightface.app import FaceAnalysis
from transformers import ViTForImageClassification
import torchvision.transforms as transforms
from torchvision.transforms import functional as TF
from scipy.signal import butter, filtfilt
from scipy.ndimage import gaussian_filter1d
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

c:\Users\Admin\anaconda3\envs\dlib_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\anaconda3\envs\dlib_env\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Admin\anaconda3\envs\dlib_env\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
# === SETUP ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
video_path = r"E:\UET\HMI\Pj\Input\Trich doan 5 (He theo thay).mp4"
model_path = r"E:\UET\HMI\Pj\models\vit_model.pth"

target_size = (224, 224)
num_classes = 12
IMG_SIZE = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

emotion_labels = ['Genuine Disgust', 'Posed Disgust', 'Genuine Happiness', 'Posed Happiness',
            'Genuine Fear', 'Posed Fear', 'Genuine Anger', 'Posed Anger',
            'Genuine Surprise', 'Posed Surprise', 'Genuine Sadness', 'Posed Sadness']
num_classes = len(emotion_labels)

# === Identity Database Setup ===
identity_db = {}
track_to_person = {}
next_person_id = 1

In [3]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [4]:
# === FUNCTIONS ===
def resize_with_aspect_ratio_padding(image_bgr, target_size=IMG_SIZE):
    h, w = image_bgr.shape[:2]
    new_h = target_size
    new_w = int(w * target_size / h)

    img_resized = cv2.resize(image_bgr, (new_w, new_h))
    delta_w = target_size - new_w

    if delta_w > 0:
        left = delta_w // 2
        right = delta_w - left
        img_padded = cv2.copyMakeBorder(img_resized, 0, 0, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    else:
        start_w = (new_w - target_size) // 2
        img_padded = img_resized[:, start_w:start_w + target_size]

    return img_padded

def eulerian_magnify_video(face_frames, freq_min=0.7, freq_max=4.0, amplification=50, fs=30):
    def temporal_bandpass_filter(signal, freq_min, freq_max, fs):
        nyquist = 0.5 * fs
        low = freq_min / nyquist
        high = freq_max / nyquist
        b, a = butter(2, [low, high], btype='band')
        return filtfilt(b, a, signal, axis=0)

    face_stack = np.stack(face_frames).astype(np.float32)
    yuv_stack = np.array([cv2.cvtColor(f, cv2.COLOR_BGR2YUV) for f in face_stack])
    y, u, v = yuv_stack[..., 0], yuv_stack[..., 1], yuv_stack[..., 2]
    u_filtered = temporal_bandpass_filter(u, freq_min, freq_max, fs)
    v_filtered = temporal_bandpass_filter(v, freq_min, freq_max, fs)
    u_amplified = np.clip(u + amplification * u_filtered, 0, 255).astype(np.uint8)
    v_amplified = np.clip(v + amplification * v_filtered, 0, 255).astype(np.uint8)
    magnified = [cv2.cvtColor(cv2.merge([y[i].astype(np.uint8), u_amplified[i], v_amplified[i]]), cv2.COLOR_YUV2BGR) for i in range(len(face_frames))]
    return magnified

def calculate_optical_flow(prev_frame, next_frame):
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None,
                                        0.5, 3, 15, 3, 5, 1.2, 0)
    return flow

def calculate_motion_intensity(flow):
    mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    return np.sum(mag)

def detect_segment_start(flow_buffer, min_length=3, sigma=1, motion_threshold=300):
    if len(flow_buffer) < min_length:
        return False

    intensities = []
    for i in range(1, len(flow_buffer)):
        flow = calculate_optical_flow(flow_buffer[i-1], flow_buffer[i])
        intensity = calculate_motion_intensity(flow)
        intensities.append(intensity)

    smooth = gaussian_filter1d(intensities, sigma=sigma)
    gradients = np.gradient(smooth)

    increasing = np.sum(np.array(gradients[-min_length:]) > 0)
    avg_motion = np.mean(smooth[-min_length:])

    return increasing >= (min_length - 1) and avg_motion > motion_threshold

def detect_segment_end(current_segment, stable_count=4, sigma=1, motion_threshold=300):
    if len(current_segment) < stable_count + 1:
        return False

    intensities = []
    for i in range(1, len(current_segment)):
        flow = calculate_optical_flow(current_segment[i-1], current_segment[i])
        intensity = calculate_motion_intensity(flow)
        if intensity == 0:
            return False
        intensities.append(intensity)

    smooth = gaussian_filter1d(intensities, sigma=sigma)
    gradients = np.gradient(smooth)

    recent_grad = gradients[-stable_count:]
    recent_motion = smooth[-stable_count:]
    is_decreasing = np.sum(np.array(recent_grad) < 0) >= (stable_count - 1)
    is_low_motion = np.mean(recent_motion) < motion_threshold

    return is_decreasing or is_low_motion

def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, yB - yA) * max(0, xB - xA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-6)
    return iou


def compute_cosine_sim(a, b):
    return cosine_similarity([a], [b])[0][0]

def assign_person_id(face, frame_id, blur, threshold=0.4, max_embeddings=30):
    global next_person_id, identity_db
    if blur > 3500:
        return -1
    if next_person_id == 1:
        if face.det_score < 0.8 and blur > 2000:
            return -1
        person_id = next_person_id
        next_person_id += 1
        print(f"[DEBUG] Gán ID mới: {person_id} tại frame {frame_id}")
        identity_db[person_id] = {
            'embeddings': [face.embedding.reshape(1, -1)],
            'first_seen': frame_id,
            'last_seen': frame_id,
            'bbox': face.bbox
        }
        return person_id
    emb = face.embedding.reshape(1, -1)
    best_id, best_sim = None, 0
    for pid, info in identity_db.items():
        embeddings = np.array(info['embeddings'])
        sims = [cosine_similarity(emb, e) for e in embeddings]
        avg_sim = sum(sims) / len(sims) if sims else 0

        if avg_sim > best_sim:
            best_sim = avg_sim
            best_id = pid
    if blur > 1200:
        data = identity_db.get(best_id, None)
        if data is not None:
            prev_frame_id = data['last_seen']
            iou = compute_iou(data['bbox'], face.bbox)
            if best_sim >= 0.3 and iou > 0.4:
                return best_id
            else:
                return -1

    if best_sim >= threshold:
        if best_id is None:
            return -1
        identity_db[best_id]['last_seen'] = frame_id
        if blur < 1000:
            identity_db[best_id]['embeddings'].append(emb)
        if len(identity_db[best_id]['embeddings']) > max_embeddings:
            identity_db[best_id]['embeddings'].pop(0)  

        return best_id
    
    if next_person_id == 3:
        if best_id is None:
            return -1
        identity_db[best_id]['last_seen'] = frame_id
        identity_db[best_id]['embeddings'].append(emb)
        if len(identity_db[best_id]['embeddings']) > max_embeddings:
            identity_db[best_id]['embeddings'].pop(0)
        return best_id

    person_id = next_person_id
    next_person_id += 1

    identity_db[person_id] = {
        'embeddings': [emb],
        'first_seen': frame_id,
        'last_seen': frame_id,
        'bbox': face.bbox
    }
    return person_id


    
def align_face(frame, landmarks, output_size=(112, 112)):
    dst = np.array([
        [38.2946, 51.6963],
        [73.5318, 51.5014],
        [56.0252, 71.7366],
        [41.5493, 92.3655],
        [70.7299, 92.2041]
    ], dtype=np.float32)

    src = np.array(landmarks, dtype=np.float32)
    M = cv2.estimateAffinePartial2D(src, dst, method=cv2.LMEDS)[0]
    aligned = cv2.warpAffine(frame, M, output_size, borderValue=0.0)
    return aligned

In [5]:
def assign_person_1_id(face, frame_id, blur, threshold=0.4, max_embeddings=30):
    global next_person_id, identity_db
    if blur > 3500:
        return -1
    return 1

In [6]:
# === LOAD MODELS ===
app = FaceAnalysis(name="buffalo_l") 
app.prepare(ctx_id=0) 
tracker = DeepSort(max_age= 30, n_init=3, nn_budget=100)

model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")
model.classifier = torch.nn.Linear(in_features=768, out_features=num_classes, bias=True)
model.to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

c:\Users\Admin\anaconda3\envs\dlib_env\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Admin/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Admin/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Admin/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Admin/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Admin/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

c:\Users\Admin\anaconda3\envs\dlib_env\lib\site-packages\deep_sort_realtime\embedder\embedder_pytorch.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_st

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [7]:
def calculate_blur_score(image):
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    variance = laplacian.var()
    return variance

In [ ]:
# === VIDEO PROCESSING ===
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_id = 0
active_tracks = {}
flow_buffer = {}
all_segments = []
count_pid = 1
cnt = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_id += 1
    faces = app.get(frame)

    detections = []
    for face in faces:
        x1, y1, x2, y2 = face.bbox.astype(int)
        conf = face.det_score
        if conf < 0.7 or x2 - x1 < 70 or y2 - y1 < 70:
            continue

        bbox = [x1, y1, x2, y2]
        face_img = frame[y1:y2, x1:x2]
        if face_img is None or face_img.size == 0:
            continue

        blur = calculate_blur_score(face_img)
        person_id = assign_person_1_id(face, frame_id, blur)
        if person_id == -1:
            continue
        face_img = resize_with_aspect_ratio_padding(face_img, target_size=IMG_SIZE)

        # Save debug image
        DEBUG_DIR = r"E:\UET\HMI\Pj\Debug"
        save_dir = os.path.join(DEBUG_DIR, f"person_{person_id}")
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"frame_{frame_id}_blur_{blur}.jpg")
        cv2.imwrite(save_path, face_img)

        # Update active track
        if person_id in active_tracks:
            data = active_tracks[person_id]
            data['end-time'] = frame_id / fps
            data['end-frame'] = frame_id
            data['segment']['images'].append(face_img)
            data['segment']['landmarks'].append(face.kps)
            data['segment']['frame_ids'].append(frame_id)

            end_segment = detect_segment_end(data['segment']['images'])
            if end_segment:
                all_segments.append(data)
                del active_tracks[person_id]
                if person_id in flow_buffer:
                    del flow_buffer[person_id]
            elif(data['segment']['frame_ids'][-1] - data['segment']['frame_ids'][0]) >= 13:
                if len(data['segment']['images']) >= 5:
                    all_segments.append(data)
                del active_tracks[person_id]
                if person_id in flow_buffer:
                    del flow_buffer[person_id]                

        else:
            # Khởi tạo track mới
            active_tracks[person_id] = {
                'start-time': None,
                'start-frame': frame_id,
                'end-time': None,
                'end-frame': None,
                'segment': {
                    'images': [],
                    'landmarks': [],
                    'frame_ids': []
                },
                'person_id': person_id
            }

            start_segment = False
            if len(flow_buffer.get(person_id, [])) >= 2:
                start_segment = detect_segment_start(flow_buffer[person_id])
            else:
                start_segment = False

            if start_segment:
                active_tracks[person_id]['segment']['images'].append(face_img)
                active_tracks[person_id]['segment']['landmarks'].append(face.kps)
                active_tracks[person_id]['segment']['frame_ids'].append(frame_id)
                active_tracks[person_id]['start-time'] = frame_id / fps
                active_tracks[person_id]['start-frame'] = frame_id
                active_tracks[person_id]['end-time'] = frame_id / fps
                active_tracks[person_id]['end-frame'] = frame_id
            else:
                # Lưu ảnh vào flow_buffer để đợi phát hiện bắt đầu segment
                if flow_buffer.get(person_id) is None:
                    flow_buffer[person_id] = [face_img]
                else:
                    flow_buffer[person_id].append(face_img)

cap.release()

c:\Users\Admin\anaconda3\envs\dlib_env\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [ ]:
print(len(all_segments))

1148


In [ ]:
min_frames = 10
for pid in list(identity_db.keys()):
    if len(identity_db[pid]['embeddings']) < min_frames:
        print(f"[DEBUG] Xóa ID {pid} do quá ít frame ({len(identity_db[pid]['embeddings'])} < {min_frames})")

        del identity_db[pid]

        all_segments = [segment for segment in all_segments if segment['person_id'] != pid]

In [ ]:
# === XỬ LÝ CÁC ĐOẠN VIDEO ĐÃ LƯU ===
cap = cv2.VideoCapture(video_path)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
video_name = os.path.splitext(os.path.basename(video_path))[0]
play_name, excerpt_name = video_name.split("(")[0].strip(), video_name.split("(")[-1].replace(")", "").strip()

output_dir = os.path.join("Output", video_name)
apex_folder = os.path.join(output_dir, "apex_images")
segment_folder = os.path.join(output_dir, "segment_images")
os.makedirs(apex_folder, exist_ok=True)
os.makedirs(segment_folder, exist_ok=True)
results = []
image_label_list = []
count = 0
for data in all_segments:
    segment = data['segment']
    images = segment['images']
    landmarks = segment['landmarks']
    frame_ids = segment['frame_ids']

    if len(images) < 2:
        continue
    
    # segment = eulerian_magnify_video(segment, fs=fps)
    onset_frame_id = frame_ids[0]
    onset_time = onset_frame_id / fps

    apex_idx, highest = 0, -1
    for i, face_img in enumerate(images):
        if i == len(images) - 1:
            continue
        flow = calculate_optical_flow(images[0], face_img)
        intensity = calculate_motion_intensity(flow)
        if intensity > highest:
            highest = intensity
            apex_idx = i
    
    apex_frame_id = frame_ids[apex_idx]
    apex_time = apex_frame_id / fps
    offset_frame_id = frame_ids[-1]
    offset_time = offset_frame_id / fps
    apex_face = images[apex_idx]

    apex_filename = f"person_{data['person_id']}_apex_frame_{apex_frame_id}.jpg"
    apex_image_path = os.path.join(apex_folder, apex_filename)
    apex_face_pil = Image.fromarray(cv2.cvtColor(apex_face, cv2.COLOR_BGR2RGB))
    apex_face_pil.save(apex_image_path)
    
    
    try:
        # aligned = align_face(apex_face, landmarks[apex_idx], output_size=(IMG_SIZE, IMG_SIZE))
        input_tensor = transform(cv2.cvtColor(apex_face, cv2.COLOR_BGR2RGB)).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(input_tensor)
            pred_idx = torch.argmax(output.logits, dim=1).item()
            emotion = emotion_labels[pred_idx]
    except:
        emotion = "Unknown"
    
    image_folder = os.path.join(output_dir, "images")
    os.makedirs(image_folder, exist_ok=True)
    
    for idx, (face_img, frame_id) in enumerate(zip(images, frame_ids)):
        img_file_name = f"person_{data['person_id']}_frame_{frame_id}.jpg"
        img_path = os.path.join(segment_folder, img_file_name)
        face_pil = Image.fromarray(cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB))
        face_pil.save(img_path)

    
    
    results.append({
        'person_id': data['person_id'],
        'onset_frame_id': onset_frame_id,
        'onset_time': onset_time,
        'apex_frame_id': apex_frame_id,
        'apex_time': apex_time,
        'offset_frame_id': offset_frame_id,
        'offset_time': offset_time,
        'emotion': emotion
    })

In [ ]:
# === SAVE FILES ===
with open(os.path.join(output_dir, f"{video_name}_labels.csv"), 'w', newline='', encoding='utf-8') as f:
    csv.writer(f).writerows([["filename", "label"]] + image_label_list)

# Save JSON
with open(os.path.join(output_dir, "emotion_results.json"), 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=4)

# Save full CSV
with open(os.path.join(output_dir, "emotion_results.csv"), 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow([
        'Play Name', 'Excerpt', 'Person ID',
        'Onset Frame ID', 
        'Apex Frame ID', 
        'Offset Frame ID',
        'Emotion'
    ])
    for r in results:
        writer.writerow([
            excerpt_name, play_name, r['person_id'],
            r['onset_frame_id'], 
            r['apex_frame_id'],
            r['offset_frame_id'],
            r['emotion']
        ])
